# Toy Radix Benders by Proximal Bundle dual decomposition

In [1]:
PREVENT_ZERO = True
REG_WEIGHT = 0. #1e-4    # Regularization weight
MAX_NONZERO  = None #48*2
CUT_STRATEGY = 'mw'  #'default'

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
mdl_ref = ijomc

ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

# N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

0.873921506968


In [4]:
df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

# 0) Load changed keffs to reduce binary vars

In [5]:
import json

with open('/home/laurence/ME/data/dynamicME/kfit_changed.json') as f:
    kfit_changed = json.load(f)

In [6]:
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel
from dynamicme.generate import copy_model

In [7]:
changed_keffs = [kv[0] for kv in kfit_changed]

# 1) Create master, subproblems

In [8]:
from dynamicme.optimize import Variable, Constraint
import numpy as np

radix = 2.
print('Radix:',radix)
#powers = np.arange(-3,4)
powers = [-1, 0, 1]
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Get the group ID from reference model
mu_id = 'BIOMASS_Ecoli_core_w_GAM'
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    mdl_ind = cond
    mdl = copy_model(ijomc, suffix='_%s'%mdl_ind)            
    opt = Optimizer(mdl)
    gap = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)
    #----------------------------------------------------
    # Now, add min abs err
    #----------------------------------------------------
    for rxn in gap.reactions:
        rxn.objective_coefficient = 0.
    mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
    sp = Variable('sp_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sn = Variable('sn_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sp.objective_coefficient = (1.-REG_WEIGHT)/(mu_meas+1) #1.
    sn.objective_coefficient = (1.-REG_WEIGHT)/(mu_meas+1) #1.
    cons = Constraint('abs_err_%s'%mdl_ind)
    cons._constraint_sense = 'E'
    cons._bound = mu_meas
    gap.add_metabolites(cons)
    gap.add_reactions([sp,sn])
    # mu - mu_meas = sp-sn
    # mu -sp + sn = mu_meas
    # min sp + sn
    sp.add_metabolites({cons:-1.})
    sn.add_metabolites({cons:1.})
    rxn_mu = gap.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    rxn_mu.add_metabolites({cons:1.})
    #----------------------------------------------------    
    dfi = df_conds[ df_conds.cond==cond]
    var_cons_dict = {}
    for rxn_ref in crowding_ref.reactions:
        if rxn_ref.id in changed_keffs:
            crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
            var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
            rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
            cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
            a0 = rxn_p.metabolites[crowding_p]
            if var_cons_dict.has_key(rxn_ref.id):
                var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
            else:
                var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        
    opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)    
    sub = LagrangeSubmodel(gap, cond)
    sub_dict[cond] = sub
    
for group_id in var_cons_dict.keys():
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):
            yid = 'binary_%s%s%s'%(group_id,k,l)
            y   = gap.reactions.get_by_id(yid)
            ### PREFER pwr=0, digit=1
            if pwr==0 and digit==1:
                y.objective_coefficient = 0.
            else:
                y.objective_coefficient = REG_WEIGHT

master = LagrangeMaster(gap)

('Radix:', 2.0)
('Powers:', [-1, 0, 1])
('Digits:', [1.0])


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:11 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [ ]:
master.add_submodels(sub_dict)

# 2) Solve

In [ ]:
# master.model.Params.OutputFlag = 0
master.print_iter = 1   # submodels take much longer
master.optimize()

        Iter          UB          LB     Best UB     Best LB         gap   relgap(%)     time(s)
           0      1e+100     -1e+100      1e+100     -1e+100      2e+100       200.0    0.000561
           1      1000.0    0.215695      1000.0    0.215695    999.7843  99.9784304    0.364567
           2    17.21569    -16.6178    17.21569    0.215695    16.99999  98.7470986    0.473801
           3    2.389143    -5.95493    2.389143    0.215695    2.173448  90.9718418    0.593003
           4    0.941963    -3.31687    0.941963    0.215695    0.726268  77.1014912    0.673928
           5    0.522120    -1.79850    0.522120    0.215695    0.306424  58.6885220    0.757638
           6    0.388494    -0.51508    0.388494    0.215695    0.172799  44.4791555    0.898898
           7    0.363492    -0.09437    0.363492    0.215695    0.147796  40.6602250    2.374444
           8    0.359281    -0.06645    0.359281    0.215695    0.143586  39.9647777    3.523038

# 3) Validate solution

In [ ]:
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
#             if abs(y)>1e-10:
#                 print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))            
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
# print('kfit_changed:',kfit_changed)

#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:    
    ijofit = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
    crowding = ijofit.metabolites.get_by_id('crowding')
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit}, combine=False)
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))

import json

with open('/home/laurence/ME/data/dynamicME/kfit_changed.json','w') as f:
    json.dump(kfit_changed,f)